### Задачи 1 и 2
Контрольная работа состоит из пяти вопросов. На каждый вопрос приведено четыре варианта ответа, один из которых правильный. Случайная величина X задаёт число правильных ответов при простом угадывании. Найдите математическое ожидание данной случайной величины. Найдите дисперсию случайной величины X.

Подсказка: постройте закон распределения случайной величины X.

In [93]:
import numpy as np

In [59]:
def combinations(n: int, k: int) -> int:
    """Число сочетаний.
    """
    
    return np.math.factorial(n) // (np.math.factorial(k) * np.math.factorial(n - k))

In [90]:
def bernuli(n,k,p):
    return combinations(n,k)*(p**k)*(1-p)**(n-k)

Таким образом наши случайные величины будут соответсвовать количеству верных ответов: $x_1 = 0$, $x_2 = 1$, $x_3 = 2$, $x_4 = 3$, $x_5 = 4$, $x_6 = 5$


По формуле Бернули найдем вероятности для каждого исхода.

In [91]:
for i in range(6):
    print(f"Вероятность {i} из 5 верных ответов, {bernuli(5,i,0.25)}")

Вероятность 0 из 5 верных ответов, 0.2373046875
Вероятность 1 из 5 верных ответов, 0.3955078125
Вероятность 2 из 5 верных ответов, 0.263671875
Вероятность 3 из 5 верных ответов, 0.087890625
Вероятность 4 из 5 верных ответов, 0.0146484375
Вероятность 5 из 5 верных ответов, 0.0009765625


Находим мат. ожидание по формуле мат. ожидания биноминального распределения: M(X) = np

In [86]:
MX = 5*0.25
MX

1.25

Т.е. при случайном выборе ответов, мы ожидаем получить 1.25 верных.

Далее ищем дисперсию по формуле дисперсии биноминального распределения D(X) = np(1-p)

In [87]:
DX = 1.25*(1-0.25)
DX

0.9375

#### Ответ: Мат ожидание = 1.25, Дисперсия = 0.9375

*Тоже самое, если посчитать, как просто распределение случайной величины.

In [97]:
vars = np.array([bernuli(5,i,0.25) for i in range(6)]) #Вероятности исходов
vars

array([0.23730469, 0.39550781, 0.26367188, 0.08789062, 0.01464844,
       0.00097656])

In [99]:
x = np.array([i for i in range(6)])  # Исходы
x

array([0, 1, 2, 3, 4, 5])

In [101]:
M = (vars*x).sum() #Мат ожидание
M

1.25

In [103]:
xstd = (x - M)**2 #Квадрат откл от мат ожидания
xstd

array([ 1.5625,  0.0625,  0.5625,  3.0625,  7.5625, 14.0625])

In [105]:
D = (xstd*vars).sum() #Дисперсия
D

0.9375

### Задача 3
Пользователь получает в среднем 10 писем со спамом на свой почтовый ящик за сутки. Найти число N, такое, что с вероятностью 0.95 пользователь получит не более N писем со спамом за текущий день.

In [106]:
def poisson_proba(k: int, lambda_: float) -> float:
    """Формула Пуассона.
    """
    
    return (lambda_ ** k) * (np.exp(-lambda_)) / np.math.factorial(k)

У нас получается, что $P(X \leq N) = \displaystyle\sum_{i = 0}^{N} \dfrac{10^{i} e^{-10}}{i!} = 0.95$

Вероятно как-то можно выразить N из формулы, но я не осилил, потому, просто написал код для итеративного решения.

In [108]:
lambda_ = 10

In [125]:
j=0
while sum(poisson_proba(k=i, lambda_=lambda_) for i in range(j)) < 0.95:
    j+=1
print(j)

16


In [124]:
sum(poisson_proba(k=i, lambda_=lambda_) for i in range(16))

0.9512595966960214

#### Ответ: N = 15

### Задача 4
Производятся выстрелы по мишени. Вероятность попадания в мишень при одном выстреле равна 0.01. Сколько выстрелов нужно сделать чтобы быть уверенным с вероятностью 0.9, что хотя бы 6 раз будет совершено попадание?

Подсказка: 1) "Вероятность попасть k раз при n выстрелах" - на какое распределение это похоже? 2) А если нам нужна вероятность P(X >= k), а не P(X = k)? 3) Здесь предстоит немножко покодить.

Будем отталкиваться от биноминального распределения. Допустим делаем первые 6 выстрелов считаем по формуле Бернули вероятность попадания, далее 